In [1]:
%cd ..

D:\Tesis\REPO\Cloud-Classification\code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [4]:
import os
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx

In [5]:
from src import config
from src import engine
from src import utils

Using backend: pytorch


In [6]:
from recipe.utils import *

In [7]:
import random
from random import sample

In [8]:
torch.manual_seed(7)
np.random.seed(7)
random.seed(7)

---

In [9]:
def get_targets(paths):
        return list(map(int,list(map(int,[os.path.basename(x).split('_')[0] 
                                          for x in paths]))))

In [46]:
def sample_paths(df, n=2, col='target', random_state = 7):
    
    paths = list()
    
    for key, sub_df in df.groupby(col):
        paths+= sub_df.sample(n=n, random_state=random_state)['path'].tolist()
        
    return paths

---

# 1. Leer df train y funcion de sampleo por clase

In [11]:
paths = utils.get_gcd_paths('D:\Tesis\REPO','train')
targets = get_targets(paths)

In [12]:
df_train = pd.DataFrame({'path':paths, 'target':targets})

# 2. Inferencia en batch de test con distintos pivots

## 2.1 test_loader

In [14]:
path_test_images = utils.get_gcd_paths('D:\Tesis\REPO','test')

test_dataset = GCD(path_test_images, resize=256)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=8,
        shuffle=False,
    )

In [15]:
len(test_dataset)

9010

## 2.2 Construir grafo con imagenes pivot

In [109]:
device='cuda'

model = GraphClassifier(7).to(device)
model.load_state_dict(torch.load(f'recipe/graph_params_AUGMENTATION.pt', map_location = 'cuda'))

model.eval()

GraphClassifier(
  (cnn): CNNExtractor(
    (cnn): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
     

In [110]:
def get_pivot_deep_features(train_paths, model, device):
    
    ## 1. train data loader
    pivot_dataset = GCD(train_paths, resize=256)
    pivot_loader = torch.utils.data.DataLoader(
            pivot_dataset,
            batch_size=32,
            num_workers=8,
            shuffle=False,
        )
    
    
    ## 2. Append deep features
    pivot_features = list()
        
    for data in pivot_loader:
        with torch.no_grad():
            deep_features = model.get_deep_features(data['images'].to(device)).cpu()
            pivot_features.append(deep_features)
        
    return torch.cat(pivot_features, dim=0)

In [111]:
pivot_paths = sample_paths(df_train, n=2)

In [112]:
pivot_features = get_pivot_deep_features(pivot_paths, model, device)

In [113]:
pivot_features.shape

torch.Size([14, 2048])

## evaluacion

In [ ]:

print('1. Reading pivot paths')
pivot_paths = sample_paths(df_train, n=8, random_state=10)

# pivot_paths = [
#     'D:\\Tesis\\REPO\\GCD/train/1_cumulus/1_cumulus_000082.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/1_cumulus/1_cumulus_000127.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/2_altocumulus/2_altocumulus_000003.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/2_altocumulus/2_altocumulus_000028.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/3_cirrus/3_cirrus_000035.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/3_cirrus/3_cirrus_000115.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/4_clearsky/4_clearsky_000004.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/4_clearsky/4_clearsky_000055.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/5_stratocumulus/5_stratocumulus_000003.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/5_stratocumulus/5_stratocumulus_000069.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/6_cumulonimbus/6_cumulonimbus_000011.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/6_cumulonimbus/6_cumulonimbus_000059.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/7_mixed/7_mixed_000006.jpg',
#     'D:\\Tesis\\REPO\\GCD/train/7_mixed/7_mixed_000039.jpg',
# ]



print('2. Calculating pivot deep features')
pivot_features = get_pivot_deep_features(pivot_paths, model, device)


test_targets = list()
test_predictions = list()

print('3. Calculating predictions')
for data in tqdm(test_loader):
    pred_img = data['images']
    
    pred_feature = model.get_deep_features(pred_img.to(device)).cpu()
    
    #print((torch.cat([pivot_features, pred_feature], dim=0)[-1] == pred_feature).all())
    
    pred_images = torch.cat([pivot_features, pred_feature], dim=0).to(device)
    
    with torch.no_grad():
        preds = torch.argmax(F.softmax(model.message_passing(pred_images).cpu(),dim=1), dim=1)
        #print(preds.shape)
        #print(preds[-1].item())
        #print(data['targets'].item())
        
        test_predictions.append(preds[-1].item())
        test_targets.append(data['targets'].item())

1. Reading pivot paths
2. Calculating pivot deep features


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(test_targets, test_predictions)

In [ ]:
r = np.array(test_targets)[(np.array(test_targets) == np.array(test_predictions))]

In [ ]:
pd.Series(r).value_counts()/pd.Series(np.array(test_targets)).value_counts()